In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

In [3]:
device

'cpu'

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [6]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)
logits

tensor([[-0.0820,  0.1174, -0.0210, -0.0581, -0.0338, -0.0094,  0.0303, -0.0214,
         -0.0784,  0.0425]], grad_fn=<AddmmBackward0>)

In [7]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.0930, 0.1135, 0.0989, 0.0953, 0.0976, 0.1000, 0.1041, 0.0988, 0.0934,
         0.1054]], grad_fn=<SoftmaxBackward0>)

In [8]:
y_pred = pred_probab.argmax(dim=1)
y_pred

tensor([1])

In [9]:
input_image = torch.rand(3, 28, 28)

In [10]:
input_image.size()

torch.Size([3, 28, 28])

In [11]:
flatten = nn.Flatten()

In [12]:
flat_image = flatten(input_image)

In [13]:
flat_image.size()

torch.Size([3, 784])

In [16]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
hidden1

tensor([[ 0.3108,  0.1250, -0.5913,  0.1164, -0.4526,  0.2968,  0.6309, -0.0724,
          0.4285, -0.3859, -0.1616, -0.5827,  0.1018, -0.3767,  0.6551,  0.2134,
          0.1735,  0.3379, -0.1421, -0.1376],
        [ 0.2278,  0.0118, -0.5480, -0.2559, -0.9448,  0.2823,  0.5343,  0.3517,
         -0.2452, -0.0301,  0.4673, -0.5408, -0.0557, -0.6136,  0.3921,  0.2963,
          0.0475,  0.2411, -0.3174,  0.0200],
        [ 0.4822,  0.1995, -0.4581, -0.1415, -0.2256,  0.2277,  0.6457,  0.1539,
         -0.0807, -0.1445,  0.5081, -0.5537,  0.1241, -0.5617,  0.5178,  0.1824,
         -0.3827,  0.3057, -0.3588,  0.3898]], grad_fn=<AddmmBackward0>)

In [18]:
hidden1.size()

torch.Size([3, 20])

In [19]:
hidden1

tensor([[ 0.3108,  0.1250, -0.5913,  0.1164, -0.4526,  0.2968,  0.6309, -0.0724,
          0.4285, -0.3859, -0.1616, -0.5827,  0.1018, -0.3767,  0.6551,  0.2134,
          0.1735,  0.3379, -0.1421, -0.1376],
        [ 0.2278,  0.0118, -0.5480, -0.2559, -0.9448,  0.2823,  0.5343,  0.3517,
         -0.2452, -0.0301,  0.4673, -0.5408, -0.0557, -0.6136,  0.3921,  0.2963,
          0.0475,  0.2411, -0.3174,  0.0200],
        [ 0.4822,  0.1995, -0.4581, -0.1415, -0.2256,  0.2277,  0.6457,  0.1539,
         -0.0807, -0.1445,  0.5081, -0.5537,  0.1241, -0.5617,  0.5178,  0.1824,
         -0.3827,  0.3057, -0.3588,  0.3898]], grad_fn=<AddmmBackward0>)

In [20]:
hidden1 = nn.ReLU()(hidden1)
hidden1

tensor([[0.3108, 0.1250, 0.0000, 0.1164, 0.0000, 0.2968, 0.6309, 0.0000, 0.4285,
         0.0000, 0.0000, 0.0000, 0.1018, 0.0000, 0.6551, 0.2134, 0.1735, 0.3379,
         0.0000, 0.0000],
        [0.2278, 0.0118, 0.0000, 0.0000, 0.0000, 0.2823, 0.5343, 0.3517, 0.0000,
         0.0000, 0.4673, 0.0000, 0.0000, 0.0000, 0.3921, 0.2963, 0.0475, 0.2411,
         0.0000, 0.0200],
        [0.4822, 0.1995, 0.0000, 0.0000, 0.0000, 0.2277, 0.6457, 0.1539, 0.0000,
         0.0000, 0.5081, 0.0000, 0.1241, 0.0000, 0.5178, 0.1824, 0.0000, 0.3057,
         0.0000, 0.3898]], grad_fn=<ReluBackward0>)

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

In [22]:
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits

tensor([[ 0.2015, -0.0558,  0.1656, -0.1868, -0.3003, -0.0374,  0.2664,  0.0190,
         -0.1616,  0.1468],
        [-0.0318, -0.0208,  0.2609, -0.2192, -0.2681, -0.0871,  0.1336, -0.0702,
         -0.1504,  0.1280],
        [ 0.1147, -0.0190,  0.2860, -0.1281, -0.2209, -0.0238,  0.3104,  0.0488,
         -0.1837,  0.1688]], grad_fn=<AddmmBackward0>)

In [23]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [24]:
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [25]:
[(name, param.size(), param[:2]) for name, param in model.named_parameters()]

[('linear_relu_stack.0.weight',
  torch.Size([512, 784]),
  tensor([[ 0.0075, -0.0189,  0.0351,  ..., -0.0184,  0.0304, -0.0063],
          [ 0.0091, -0.0080,  0.0065,  ..., -0.0242, -0.0098,  0.0143]],
         grad_fn=<SliceBackward0>)),
 ('linear_relu_stack.0.bias',
  torch.Size([512]),
  tensor([ 0.0088, -0.0286], grad_fn=<SliceBackward0>)),
 ('linear_relu_stack.2.weight',
  torch.Size([512, 512]),
  tensor([[-0.0239, -0.0093,  0.0117,  ...,  0.0305, -0.0258,  0.0385],
          [-0.0279,  0.0042, -0.0414,  ...,  0.0182, -0.0166,  0.0138]],
         grad_fn=<SliceBackward0>)),
 ('linear_relu_stack.2.bias',
  torch.Size([512]),
  tensor([ 0.0231, -0.0270], grad_fn=<SliceBackward0>)),
 ('linear_relu_stack.4.weight',
  torch.Size([10, 512]),
  tensor([[ 0.0374,  0.0404,  0.0406,  ..., -0.0223, -0.0315,  0.0069],
          [-0.0344,  0.0307, -0.0224,  ...,  0.0106,  0.0095,  0.0435]],
         grad_fn=<SliceBackward0>)),
 ('linear_relu_stack.4.bias',
  torch.Size([10]),
  tensor([-0.03